In [2]:
iris = load('iris.data');
iris_features = iris(:,1:end-1);
iris_classes = iris(:,end);

<center>
    
# Aprendizagem de Máquina (ES456/2018-2)
## Lista de exercícios 2
### 2018-11-20
    
</center>

1) Utilizando a base de dados Iris, disponível no link abaixo, sem preprocessamento e a linguagem de programação de sua preferência, implemente e avalie o desempenho dos seguintes classificadores:
<br />
<br />
a) Perceptron (4 entradas e 3 saídas, função de ativação tipo sigmóide)
<br />
b) Perceptron (4 entradas e 3 saídas, função de ativação tipo ReLu)
<br />
c) MLP (4 entradas, 10 nodos na camada intermediária e 3 na camada de saída, função de ativação do sigmóide)
<br />
d) MLP (4 entradas, 10 nodos na camada intermediária e 3 na camada de saída, função de ativação tipo ReLu)
<br />
<br />
OBS: Utilize o gradiente descendente estocástico como método de treinamento

Para a função de ativação do tipo sigmoide:
$$f(z) = \frac{1}{1 + e^{-z}}$$

Analisando a função, vê-se que $f(z) \geq 0.5$ quando $z \geq 0$ e $f(z) < 0.5$ quando $z < 0$

In [3]:
function s = sigmoid(z)
    s = 1 ./ (1 + e.^(-z));
endfunction

function r = relu(z)
    r = max(0, z);
endfunction

function p = predict(features, weights, activation)
    features = [ones(size(features, 1), 1) features];
    activation_arg = features * weights';
    p = activation(activation_arg);
endfunction

function a = add_one_columns(matrix)
    a = [ones(size(matrix, 1), 1) matrix];
endfunction

$$J(W) = \frac{1}{2m} \sum_{i = 1}^{m}(p(x^{(i)}) - y^{(i)})^2$$
$$w_j := w_j - \alpha \frac{\delta J}{\delta w_j} = w_j - \frac{\alpha}{m} \sum_{i = 1}^{m}(p(x^{(i)}) - y^{(i)})x_{j}^{(i)}$$

In [34]:
function weights = train(train, learning_rate, n, activation)
    m = length(train);
    unq_tc = unique(train(:,end))';
    train_features = train(:,1:end-1);
    tf_one_column = add_one_columns(train_features);
    
    weights = zeros(length(unq_tc), size(train, 2));
    
    if (numel(unq_tc) == 2) 
        unq_tc = [unq_tc(1)];
    endif
    
    for i = unq_tc
        train_mv = train;
        train_mv(:,end) = train_mv(:,end) == i;
        train_mv_classes = train_mv(:,end);
            
        for j = 1:n    
            p = predict(train_features, weights(i+1,:), activation);
            dJ = (p - train_mv_classes)' *  tf_one_column;
            weights(i+1,:) = weights(i+1,:) - ((learning_rate/m) * dJ);
        endfor
        
    endfor
endfunction

function [train, test] = train_test_split(dataframe, p)
    % Mistura-se as linhas
    m = size(dataframe, 1);
    idx = randperm(m);
    dataframe = dataframe(idx,:);

    % Calcula-se o numero de elementos no conjunto de treinamento
    train_off = round(size(dataframe, 1) * p);

    % Separa-se os dados
    train = dataframe(1:train_off,:);
    test = dataframe(train_off+1:end,:);
endfunction

dataset = iris;

In [81]:
disp("1a)")

% Separa-se os dados
[train_data, test] = train_test_split(dataset, 0.6);
test_features = add_one_columns(test(:,1:end-1));
test_classes = test(:,end);

% Calcula-se os pesos usando sigmoide
weights = train(test, 0.02, 1000, @sigmoid)
activ_arg = test_features * weights';

% Passa-se por um selecionador de máximo
[a p] = max(sigmoid(activ_arg), [], 2);

% Calcula-se o erro
err = sum((p-1) - test_classes != 0) / sum(numel(test_classes));
printf("Taxa de erro: %.2f", err);

1a)
weights =

   0.254040   0.432980   1.199844  -1.901812  -0.863620
   0.083797  -0.096556  -0.794910   0.516812  -0.047343
  -0.485161  -0.909356  -0.964626   1.500086   0.925896

Taxa de erro: 0.05
Taxa de erro: 0.05


In [204]:
disp("1b)")

% Separa-se os dados
[train_data, test] = train_test_split(dataset, 0.6);
test_features = add_one_columns(test(:,1:end-1));
test_classes = test(:,end);

% Calcula-se os pesos usando ReLU
weights = train(train_data, 0.02, 1000, @relu)
activ_arg = test_features * weights';

% Passa-se por um selecionador de máximo
[a p] = max(relu(activ_arg), [], 2);

% Calcula-se o erro
err = sum((p-1) - test_classes != 0) / sum(numel(test_classes));
printf("Taxa de erro: %.2f", err);

1b)
weights =

   0.11148   0.13945   0.19990  -0.31837  -0.20003
   0.23882   0.16791  -0.29664   0.10058  -0.31719
  -0.16646  -0.27521  -0.32668   0.52246   0.46991

Taxa de erro: 0.05
Taxa de erro: 0.05


In [1]:
function [weights1, weights2] = train_mlp(data, weights1, weights2, alpha=0.01, n)
    m = length(data);
    train_features = data(:,1:end-1);
    train_classes = zeros(m, rows(weights2));
    
    train_classes(:,1) = data(:,end) == 0;
    train_classes(:,2) = data(:,end) == 1;
    train_classes(:,3) = data(:,end) == 2;
    
    tf_one_column = add_one_columns(train_features);
    
    % LEARN
    for i=1:n
        activation1 = tf_one_column * weights1';
        activation2 = add_one_columns((activation1));
    
        predicted = sigmoid(activation2 * weights2');
    
        delta2 = (predicted - train_classes);
        delta1 = (delta2 * weights2);
        
        dJ2 = delta2' * activation2;
        weights2 = weights2 - (alpha/m)*dJ2;
        
        dJ1 = (delta1' * tf_one_column)(2:end,:); % removendo o bias
        weights1 = weights1 - (alpha/m)*dJ1; 
    endfor
endfunction

hidden_layers = 10;
weights1 = zeros(hidden_layers, columns(iris_features) + 1);

final_layers = 3;
weights2 = zeros(final_layers, rows(weights1) + 1);

[w1, w2] = train_mlp(iris, weights1, weights2, 0.01, 500)

add_one_columns(sigmoid(add_one_columns(iris_features)*w1'))*w2'

error: 'iris_features' undefined near line 1 column 41
error: 'weights1' undefined near line 1 column 37
error: 'iris' undefined near line 1 column 20
error: 'iris_features' undefined near line 1 column 41


2) Repita as avaliações anteriores removendo o valor médio da base de dados Iris e normalizando os dados pelo desvio padrão de cada característica.

3) Implemente o algoritmo de Kohonen com um mapa de 4x4 nodos, realize uma categorização sobre conjunto de dados Iris e plote o mapa atribuindo uma cor para cada nodo para categoria.

4) Implemente uma rede RBF com bases centradas nos nodos do SOM, da questão anterior e avalie o desempenho de classificação o dataset do Iris.

5) Utilize um ADALINE para detectar a sequência binária 1101 em um canal com ruído gaussiano aditivo de $0.1$; O dataset para treinamento e teste é:
<br />
“010011011000111011100010011002002201110011003001100.500011001000011110001111”

6) Utilize uma MLP com memória para realizar a mesma detecção anterior, avalie e compare com do ADALINE